# SQL with Danny's Diner

This SQL project is based on Case #1 from [Data with Dany](https://8weeksqlchallenge.com/case-study-1/). 

## Introduction
Danny seriously loves Japanese food so in the beginning of 2021, he decides to embark upon a risky venture and opens up a cute little restaurant that sells his 3 favourite foods: sushi, curry and ramen.

Danny’s Diner is in need of your assistance to help the restaurant stay afloat - the restaurant has captured some very basic data from their few months of operation but have no idea how to use their data to help them run the business.

## Problem Statement
Danny wants to use the data to answer a few simple questions about his customers, especially about their visiting patterns, how much money they’ve spent and also which menu items are their favourite. Having this deeper connection with his customers will help him deliver a better and more personalised experience for his loyal customers.

He plans on using these insights to help him decide whether he should expand the existing customer loyalty program - additionally he needs help to generate some basic datasets so his team can easily inspect the data without needing to use SQL.

Danny has provided you with a sample of his overall customer data due to privacy issues - but he hopes that these examples are enough for you to write fully functioning SQL queries to help him answer his questions!

Danny has shared with you 3 key datasets for this case study:
- sales
- menu
- members


The ```sales``` table captures all ```customer_id``` level purchases with an corresponding ```order_date``` and ```product_id``` information for when and what menu items were ordered.

The ```menu``` table maps the ```product_id``` to the actual ```product_name``` and ```price``` of each menu item.

The ```members``` table captures the ```join_date``` when a ```customer_id``` joined the beta version of the Danny’s Diner loyalty program.

## Questions

- What is the total amount each customer spent at the restaurant?
- How many days has each customer visited the restaurant?
- What was the first item from the menu purchased by each customer?
- What is the most purchased item on the menu and how many times was it purchased by all customers?
- Which item was the most popular for each customer?
- Which item was purchased first by the customer after they became a member?
- Which item was purchased just before the customer became a member?
- What is the total items and amount spent for each member before they became a member?
- If each $1 spent equates to 10 points and sushi has a 2x points multiplier - how many points would each customer have?
- In the first week after a customer joins the program (including their join date) they earn 2x points on all items, not just sushi - how many points do customer A and B have at the end of January?


## Build database and query function

In [7]:
import sqlite3
from sqlite3 import Error


In [4]:
conn = sqlite3.connect("diner.db")


In [5]:
def create_connection(db_file):
    """ create a database connection to the SQLite database
        specified by db_file
    :param db_file: database file
    :return: Connection object or None
    """
    conn = None
    try:
        conn = sqlite3.connect(db_file)
        return conn
    except Error as e:
        print(e)

    return conn


In [11]:
def create_table(conn, create_table_sql):
    """ create a table from the create_table_sql statement
    :param conn: Connection object
    :param create_table_sql: a CREATE TABLE statement
    :return:
    """
    try:
        c = conn.cursor()
        c.execute(create_table_sql)
    except Error as e:
        print(e)


In [18]:
conn = create_connection('diner.db')


In [19]:
create_table_sql = "CREATE TABLE sales ('customer_id' VARCHAR(1),'order_date' DATE,'product_id' INTEGER);"

create_table(conn,create_table_sql)
    

table sales already exists


In [20]:
insert_table_sql = "INSERT INTO sales('customer_id', 'order_date', 'product_id') VALUES ('A', '2021-01-01', '1'), ('A', '2021-01-01', '2'),('A', '2021-01-07', '2'),  ('A', '2021-01-10', '3'),  ('A', '2021-01-11', '3'),  ('A', '2021-01-11', '3'),  ('B', '2021-01-01', '2'),  ('B', '2021-01-02', '2'),  ('B', '2021-01-04', '1'),  ('B', '2021-01-11', '1'),  ('B', '2021-01-16', '3'),  ('B', '2021-02-01', '3'),  ('C', '2021-01-01', '3'),  ('C', '2021-01-01', '3'),('C', '2021-01-07', '3');"
    
create_table(conn,insert_table_sql)

In [37]:

create_menu = 'CREATE TABLE menu ("product_id" INTEGER,"product_name" VARCHAR(5),  "price" INTEGER);'

create_table(conn,create_menu)

insert_menu = "INSERT INTO menu ('product_id', 'product_name', 'price') VALUES  ('1', 'sushi', '10'),  ('2', 'curry', '15'),  ('3', 'ramen', '12');"

create_table(conn,insert_menu)



create_member = "CREATE TABLE members ( 'customer_id' VARCHAR(1), 'join_date' DATE);"

create_table(conn,create_member)

insert_member = "INSERT INTO members ('customer_id', 'join_date') VALUES ('A', '2021-01-07'),  ('B', '2021-01-09');"

create_table(conn,insert_member)

In [32]:
def run_query(conn,query):
    cur = conn.cursor()
    cur.execute(query)
    rows = cur.fetchall()

    for row in rows:
        print(row)

In [38]:
query = "SELECT join_date FROM members;"

run_query(conn,query)

    

('2021-01-07',)
('2021-01-09',)


### What is the total amount each customer spent at the restaurant?

In [ ]:
SELECT s.customer_id, SUM(price)
FROM sales as s
LEFT JOIN menu as m
ON s.product_id = m.product_id
GROUP BY s.customer_id;

In [42]:
query = "SELECT s.customer_id, SUM(price) FROM sales as s LEFT JOIN menu as m ON s.product_id = m.product_id GROUP BY s.customer_id"

run_query(conn,query)


('A', 76)
('B', 74)
('C', 36)


### How many days has each customer visited the restaurant?


In [ ]:
SELECT customer_id, COUNT(DISTINCT(order_date))
FROM sales
GROUP BY customer_id;

In [48]:
query = "SELECT customer_id, COUNT(DISTINCT(order_date)) FROM sales GROUP BY customer_id;"

run_query(conn,query)


('A', 4)
('B', 6)
('C', 2)


### What was the first item from the menu purchased by each customer?

The tricky part of this question is when a customers first purchase included two items. We'll need to include both of the items. Let's use a CTE to create a ranked list by customer ID and order date, then we can select where rank = 1. 


In [ ]:
WITH Rank AS
(
SELECT S.customer_id, 
       M.product_name, 
       S.order_date,
       DENSE_RANK() OVER (PARTITION BY S.Customer_ID Order by S.order_date) as rank
FROM menu AS m
JOIN sales AS s
On m.product_id = s.product_id
group by S.customer_id, M.product_name,S.order_date
)
Select Customer_id, product_name
From Rank
Where rank = 1

In [51]:
query = "WITH Rank AS (SELECT S.customer_id, m.product_name, S.order_date, DENSE_RANK() OVER (PARTITION BY S.Customer_ID Order by S.order_date) as rank FROM menu AS m JOIN sales AS s On m.product_id = s.product_id group by S.customer_id, M.product_name,S.order_date) Select Customer_id, product_name From Rank Where rank = 1;"

run_query(conn,query)



('A', 'curry')
('A', 'sushi')
('B', 'curry')
('C', 'ramen')
